<a href="https://colab.research.google.com/github/aneez-hassan/PRODIGY_GA_01/blob/main/gpt2_text_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets torch --quiet


In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments

# Load tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")


In [ ]:
from google.colab import files
uploaded = files.upload()


In [ ]:
from datasets import load_dataset

# Load your dataset from a text file
dataset = load_dataset('text', data_files={'train': 'data.txt'})

# Example access
print(dataset['train'][0])


In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"


In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=2,
    save_steps=500,
    save_total_limit=2,
    prediction_loss_only=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
)

trainer.train()


In [ ]:
trainer.save_model("./fine_tuned_gpt2")
tokenizer.save_pretrained("./fine_tuned_gpt2")


In [ ]:
from transformers import pipeline

generator = pipeline("text-generation", model="./fine_tuned_gpt2")
prompt = "Once upon a time"
result = generator(prompt, max_length=100, num_return_sequences=1)
print(result[0]['generated_text'])


In [ ]:
!zip -r fine_tuned_gpt2.zip ./fine_tuned_gpt2
files.download("fine_tuned_gpt2.zip")


In [ ]:
pip install torch transformers


In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Path to your extracted fine_tuned_gpt2 folder
model_path = "fine_tuned_gpt2"

# Load model and tokenizer
tokenizer = GPT2Tokenizer.from_pretrained(model_path)
model = GPT2LMHeadModel.from_pretrained(model_path)


In [ ]:
prompt = "Once upon a time"
inputs = tokenizer(prompt, return_tensors="pt")

output = model.generate(
    inputs["input_ids"],
    max_length=50,        # total tokens (prompt + generated)
    temperature=0.7,      # randomness (lower = more focused)
    top_p=0.9,            # nucleus sampling
    do_sample=True        # enables sampling instead of greedy search
)

print(tokenizer.decode(output[0], skip_special_tokens=True))
